In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [10]:
import time
import platform
import psutil
import threadpoolctl
import sklearn
import numpy as np
import pandas as pd
import lightgbm
import skforecast
from packaging.version import parse as parse_version

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from skforecast.utils import check_predict_input
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
from skforecast.model_selection._validation import _backtesting_forecaster

%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
# !pip install pyinstrument
# !pip install line_profiler
# !pip install lightgbm==4.6.0
# !pip install -U scikit-learn==1.6.1 lightgbm==4.6.0 pandas==2.2.3 numpy==2.1.3

# conda create -n skforecast_benchmark_3_12_9 python=3.12.9

In [3]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"lightgbm version: {lightgbm.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")
print("Threadpools:", threadpoolctl.threadpool_info())
# Info BLAS detallada (NumPy >=1.26)
try:
    import numpy.__config__ as c
    print(c.get_info('blas_opt_info'))
except Exception as e:
    print("No BLAS info:", e)

Python version: 3.12.11
skforecast version: 0.20.0
scikit-learn version: 1.8.0
lightgbm version: 4.6.0
pandas version: 2.3.1
numpy version: 2.1.3
Computer network name: ITES015-NB0029
Processor type: Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type: Windows-11-10.0.26100-SP0
Operating system: Windows
Operating system release: 11
Operating system version: 10.0.26100
Number of physical cores: 8
Number of logical cores: 16
Threadpools: [{'user_api': 'blas', 'internal_api': 'openblas', 'num_threads': 16, 'prefix': 'libscipy_openblas', 'filepath': 'C:\\Users\\jaesc2\\Miniconda3\\envs\\skforecast_py12\\Lib\\site-packages\\numpy.libs\\libscipy_openblas64_-c16e4918366c6bc1f1cd71e28ca36fc0.dll', 'version': '0.3.27', 'threading_layer': 'pthreads', 'architecture': 'SkylakeX'}, {'user_api': 'openmp', 'internal_api': 'openmp', 'num_threads': 16, 'prefix': 'vcomp', 'filepath': 'C:\\Users\\jaesc2\\Miniconda3\\envs\\skforecast_py12\\Lib\\site-packages\\sklearn\\.libs\\vcomp140.dll', '

In [4]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=exog.index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [5]:
import hashlib

def hash_dataframe(df):
    row_hashes = pd.util.hash_pandas_object(df, index=True).values
    combined_hash = hashlib.sha256(row_hashes.tobytes()).hexdigest()
    return combined_hash

hash_y = hash_dataframe(y)
hash_exog = hash_dataframe(exog)
hash_exog_prediction = hash_dataframe(exog_prediction)
print(f"Hash of y: {hash_y}")
print(f"Hash of exog: {hash_exog}")
print(f"Hash of exog_prediction: {hash_exog_prediction}")
display(y.dtypes)
display(exog.dtypes)
display(exog_prediction.dtypes)

Hash of y: 24c49add46b2b7b5e6cec07d2102457a62a3a820705050b0f2ac829b178d2d8c
Hash of exog: 10fb78eb4190dd80df8de7bff30fa2fc8e974c5da5064ef2eaf6b69a5c17cf3d
Hash of exog_prediction: b533b86d8d547d466cdb52ceba8ae4ce5261765906d976d76078c6f2cfa0c5e4


dtype('float64')

day_of_week     int32
week_of_year    int64
month           int32
dtype: object

day_of_week     int32
week_of_year    int64
month           int32
dtype: object

In [6]:
regressor = LGBMRegressor(random_state=8520, verbose=-1)
regressor = LinearRegression()
forecaster = ForecasterRecursive(
    regressor=regressor,
    lags=50,
    transformer_y=None,
    transformer_exog=None,
)
forecaster.fit(y=y, exog=exog)

C:\Users\jaesc2\AppData\Local\Temp\ipykernel_49592\1249645877.py:3: FutureWarning: The `regressor` argument is deprecated and will be removed in a future version. Please use `estimator` instead.
  forecaster = ForecasterRecursive(


## skforecast 0.17.0

In [8]:
%%timeit -n 200 -r 100

check_predict_input(
    forecaster_name  = type(forecaster).__name__,
    steps            = 100,
    is_fitted        = forecaster.is_fitted,
    exog_in_         = forecaster.exog_in_,
    index_type_      = forecaster.index_type_,
    index_freq_      = forecaster.index_freq_,
    window_size      = forecaster.window_size,
    last_window      = forecaster.last_window_,
    exog             = exog_prediction,
    exog_names_in_   = forecaster.exog_names_in_,
    interval         = None
)

439 μs ± 72.4 μs per loop (mean ± std. dev. of 100 runs, 200 loops each)


In [9]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(forecaster, steps, exog):
    check_predict_input(
    forecaster_name  = type(forecaster).__name__,
    steps            = steps,
    is_fitted        = forecaster.is_fitted,
    exog_in_         = forecaster.exog_in_,
    index_type_      = forecaster.index_type_,
    index_freq_      = forecaster.index_freq_,
    window_size      = forecaster.window_size,
    last_window      = forecaster.last_window_,
    exog             = exog,
    exog_names_in_   = forecaster.exog_names_in_,
    interval         = None
)


%lprun -f check_predict_input funt_to_profile(forecaster, 100, exog_prediction)

Timer unit: 1e-09 s

Total time: 0.00215959 s
File: /home/joaquin/Documents/GitHub/skforecast/skforecast/utils/utils.py
Function: check_predict_input at line 776

Line #      Hits         Time  Per Hit   % Time  Line Contents
   776                                           def check_predict_input(
   777                                               forecaster_name: str,
   778                                               steps: int | list[int],
   779                                               is_fitted: bool,
   780                                               exog_in_: bool,
   781                                               index_type_: type,
   782                                               index_freq_: str,
   783                                               window_size: int,
   784                                               last_window: pd.Series | pd.DataFrame | None,
   785                                               last_window_exog: pd.Series | pd.DataFrame 

In [10]:
%%timeit -n 100 -r 150

forecaster._create_predict_inputs(
    steps=100, exog=exog_prediction
)

2.12 ms ± 114 μs per loop (mean ± std. dev. of 150 runs, 100 loops each)


In [11]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster._create_predict_inputs(
        steps=steps, exog=exog
    )


%lprun -f forecaster._create_predict_inputs funt_to_profile(100, exog_prediction)

Timer unit: 1e-09 s

Total time: 0.00959741 s
File: /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive.py
Function: _create_predict_inputs at line 1113

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1113                                               def _create_predict_inputs(
  1114                                                   self,
  1115                                                   steps: int | str | pd.Timestamp, 
  1116                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1117                                                   exog: pd.Series | pd.DataFrame | None = None,
  1118                                                   predict_probabilistic: bool = False,
  1119                                                   use_in_sample_residuals: bool = True,
  1120                                                   use_binned_residuals: bool = True,
  1121               

In [12]:
%%timeit -n 20 -r 10

# skforecast 0.17.0
forecaster.predict(
    steps=100, exog=exog_prediction
)

16 ms ± 445 μs per loop (mean ± std. dev. of 10 runs, 20 loops each)


In [13]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster.predict(
        steps=steps, exog=exog
    )


%lprun -f forecaster.predict funt_to_profile(100, exog_prediction)

Timer unit: 1e-09 s

Total time: 0.0785089 s
File: /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive.py
Function: predict at line 1457

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1457                                               def predict(
  1458                                                   self,
  1459                                                   steps: int | str | pd.Timestamp,
  1460                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1461                                                   exog: pd.Series | pd.DataFrame | None = None,
  1462                                                   check_inputs: bool = True
  1463                                               ) -> pd.Series:
  1464                                                   """
  1465                                                   Predict n steps ahead. It is an recursive process in which, each

In [14]:
import warnings
warnings.filterwarnings(
    "ignore", 
    message=".*X does not have valid feature names.*", 
    category=UserWarning
)

In [15]:
last_window_values=forecaster.last_window_.to_numpy().flatten()
exog_values=exog.to_numpy()

In [16]:
%%timeit -n 20 -r 20

# skforecast 0.17.0
forecaster._recursive_predict(
    steps=100,
    last_window_values=last_window_values,
    exog_values=exog_values,
    residuals=None,
    use_binned_residuals=True
)

13 ms ± 682 μs per loop (mean ± std. dev. of 20 runs, 20 loops each)


In [17]:
# skforecast 0.17.0
# ==============================================================================

import warnings
warnings.filterwarnings(
    "ignore", 
    message=".*X does not have valid feature names.*", 
    category=UserWarning
)


def funt_to_profile(steps, exog):
    forecaster._recursive_predict(
        steps=steps,
        last_window_values=forecaster.last_window_.to_numpy().flatten(),
        exog_values=exog.to_numpy(),
        residuals = None,
        use_binned_residuals=True
    )

%lprun -f forecaster._recursive_predict funt_to_profile(100, exog_prediction)

Timer unit: 1e-09 s

Total time: 0.0705477 s
File: /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive.py
Function: _recursive_predict at line 1254

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1254                                               def _recursive_predict(
  1255                                                   self,
  1256                                                   steps: int,
  1257                                                   last_window_values: np.ndarray,
  1258                                                   exog_values: np.ndarray | None = None,
  1259                                                   residuals: np.ndarray | dict[str, np.ndarray] | None = None,
  1260                                                   use_binned_residuals: bool = True,
  1261                                               ) -> np.ndarray:
  1262                                                   """
  1263              

In [18]:
import numpy as np
rng = np.random.default_rng(321)
X_test = rng.random((1, len(forecaster.lags) + exog.shape[1])).astype(np.float64)
print(X_test.flags)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [19]:
%%timeit
forecaster.regressor.predict(X_test)

110 μs ± 3.87 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
%%timeit -n 10 -r 10

# skforecast 0.16.0
cv = TimeSeriesFold(
    initial_train_size=int(len(y)/ 2),
    steps=100,
    fixed_train_size=False,
    refit=False,
)

backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    exog=exog,
    cv=cv,
    metric='mean_squared_error',
    show_progress=False,
    verbose=False
)

71.8 ms ± 1.92 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [11]:
cv = TimeSeriesFold(
        initial_train_size=int(len(y)/ 2),
        steps=100,
        fixed_train_size=False,
        refit=False,
    )

def funt_to_profile(forecaster, y, exog, cv):
   

    _backtesting_forecaster(
        forecaster=forecaster,
        y=y,
        exog=exog,
        cv=cv,
        metric='mean_squared_error',
        show_progress=False,
        verbose=False
    )

%lprun -f _backtesting_forecaster funt_to_profile(forecaster, y, exog, cv)

Timer unit: 1e-07 s

Total time: 0.496006 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_validation.py
Function: _backtesting_forecaster at line 34

Line #      Hits         Time  Per Hit   % Time  Line Contents
    34                                           def _backtesting_forecaster(
    35                                               forecaster: object,
    36                                               y: pd.Series,
    37                                               metric: str | Callable | list[str | Callable],
    38                                               cv: TimeSeriesFold,
    39                                               exog: pd.Series | pd.DataFrame | None = None,
    40                                               interval: float | list[float] | tuple[float] | str | object | None = None,
    41                                               interval_method: str = 'bootstrapping',
    42                                               n